<a href="https://colab.research.google.com/github/Data-Creater-Atlas/Data-Atlas/blob/sungjoon/Mission_1_(9_12).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# 원본 데이터가 있는 경로
ROOT = "/content/drive/MyDrive/Data Camp/53.대기오염_배출원_공간_분포_데이터/3.개방데이터/1.데이터"

# 원천/라벨 소스
TRAIN_IMG_SOURCES = [f"{ROOT}/Training/01.원천데이터/TS_KS.zip", f"{ROOT}/Training/01.원천데이터/TS_KS"]
TRAIN_LBL_SOURCES = [f"{ROOT}/Training/02.라벨링데이터/TL_KS_BBOX.zip", f"{ROOT}/Training/02.라벨링데이터/TL_KS_BBOX"]
VAL_IMG_SOURCES   = [f"{ROOT}/Validation/01.원천데이터/VS_KS.zip", f"{ROOT}/Validation/01.원천데이터/VS_KS"]
VAL_LBL_SOURCES   = [f"{ROOT}/Validation/02.라벨링데이터/VL_KS_BBOX.zip", f"{ROOT}/Validation/02.라벨링데이터/VL_KS_BBOX"]

# 작업 베이스
BASE = "/content/ks_chimney_yolo"

import os
for p in [
    f"{BASE}/raw/train/images", f"{BASE}/raw/train/labels_json",
    f"{BASE}/raw/val/images",   f"{BASE}/raw/val/labels_json",
    f"{BASE}/yolo/images/train",f"{BASE}/yolo/images/val",
    f"{BASE}/yolo/labels/train",f"{BASE}/yolo/labels/val",
]:
    os.makedirs(p, exist_ok=True)

print("BASE:", BASE)

BASE: /content/ks_chimney_yolo


In [3]:
# === 1. Collect & normalize images; copy JSONs ===
import os, glob, shutil, zipfile
from PIL import Image

IMG_EXTS = {".jpg",".jpeg",".png",".tif",".tiff",".bmp",".JPG",".JPEG",".PNG",".TIF",".TIFF",".BMP"}

def _iter_existing(paths):
    for p in paths:
        if os.path.exists(p):
            yield p

def _copy_images_any(src, out_dir):
    """src가 폴더면 재귀 수집, ZIP이면 풀어서 처리. 모든 이미지를 RGB JPG로 저장."""
    os.makedirs(out_dir, exist_ok=True)
    cnt = 0
    if os.path.isdir(src):
        files = glob.glob(os.path.join(src, "**", "*"), recursive=True)
        for fp in files:
            ext = os.path.splitext(fp)[1]
            if ext in IMG_EXTS and os.path.isfile(fp):
                try:
                    im = Image.open(fp)
                    if im.mode != "RGB":
                        im = im.convert("RGB")
                    stem = os.path.splitext(os.path.basename(fp))[0]
                    im.save(os.path.join(out_dir, f"{stem}.jpg"), quality=92)
                    cnt += 1
                except Exception:
                    pass
    elif src.lower().endswith(".zip"):
        tmp = os.path.join(BASE, "_tmp_img")
        shutil.rmtree(tmp, ignore_errors=True)
        os.makedirs(tmp, exist_ok=True)
        with zipfile.ZipFile(src) as z:
            z.extractall(tmp)
        cnt += _copy_images_any(tmp, out_dir)
        shutil.rmtree(tmp, ignore_errors=True)
    return cnt

def _copy_jsons_any(src, out_dir):
    os.makedirs(out_dir, exist_ok=True)
    cnt = 0
    if os.path.isdir(src):
        for fp in glob.glob(os.path.join(src, "**", "*.json"), recursive=True):
            try:
                shutil.copy2(fp, os.path.join(out_dir, os.path.basename(fp)))
                cnt += 1
            except:
                pass
    elif src.lower().endswith(".zip"):
        tmp = os.path.join(BASE, "_tmp_lbl")
        shutil.rmtree(tmp, ignore_errors=True)
        os.makedirs(tmp, exist_ok=True)
        with zipfile.ZipFile(src) as z:
            z.extractall(tmp)
        for fp in glob.glob(os.path.join(tmp, "**", "*.json"), recursive=True):
            try:
                shutil.copy2(fp, os.path.join(out_dir, os.path.basename(fp)))
                cnt += 1
            except:
                pass
        shutil.rmtree(tmp, ignore_errors=True)
    return cnt

# 수집
ti=tl=vi=vl=0
for s in _iter_existing(TRAIN_IMG_SOURCES): ti += _copy_images_any(s, f"{BASE}/raw/train/images")
for s in _iter_existing(TRAIN_LBL_SOURCES): tl += _copy_jsons_any(s, f"{BASE}/raw/train/labels_json")
for s in _iter_existing(VAL_IMG_SOURCES):   vi += _copy_images_any(s, f"{BASE}/raw/val/images")
for s in _iter_existing(VAL_LBL_SOURCES):   vl += _copy_jsons_any(s, f"{BASE}/raw/val/labels_json")

def _cnt_jpg(d): return len(glob.glob(os.path.join(d, "*.jpg")))
def _cnt_json(d): return len(glob.glob(os.path.join(d, "**", "*.json"), recursive=True))

print(f"[COUNT] train images: {_cnt_jpg(f'{BASE}/raw/train/images')}, train json: {_cnt_json(f'{BASE}/raw/train/labels_json')}")
print(f"[COUNT]   val images: {_cnt_jpg(f'{BASE}/raw/val/images')},   val json: {_cnt_json(f'{BASE}/raw/val/labels_json')}")


[COUNT] train images: 8062, train json: 8052
[COUNT]   val images: 1006,   val json: 1006


In [4]:
# === 2. JSON -> YOLO labels ===
import json, tqdm, shutil, os, glob

def _xywh_from_rect(rect, W, H):
    # dict: {"x","y","width","height"} or {"xmin","ymin","xmax","ymax"}
    if isinstance(rect, dict):
        if all(k in rect for k in ("x","y","width","height")):
            x,y,w,h = float(rect["x"]),float(rect["y"]),float(rect["width"]),float(rect["height"])
            return x,y,max(1.0,w),max(1.0,h)
        if all(k in rect for k in ("xmin","ymin","xmax","ymax")):
            xmin,ymin,xmax,ymax = map(float,[rect["xmin"],rect["ymin"],rect["xmax"],rect["ymax"]])
            return xmin, ymin, max(1.0,xmax-xmin), max(1.0,ymax-ymin)
    # list/tuple len 4: [x,y,w,h] or [xmin,ymin,xmax,ymax]
    if isinstance(rect,(list,tuple)) and len(rect)==4:
        a,b,c,d = map(float, rect)
        if c>a and d>b:  # treat as [xmin,ymin,xmax,ymax]
            return a,b,max(1.0,c-a),max(1.0,d-b)
        return a,b,max(1.0,c),max(1.0,d)
    raise ValueError("unsupported rect")

def _yolo_line(x,y,w,h,W,H,cls=0):
    # clip + normalize
    x = max(0,min(x,W)); y = max(0,min(y,H))
    w = max(1,min(w,W-x)); h = max(1,min(h,H-y))
    cx, cy = (x+w/2)/W, (y+h/2)/H
    return f"{cls} {cx:.6f} {cy:.6f} {w/W:.6f} {h/H:.6f}"

def _match_jpg(raw_dir, file_name):
    stem = os.path.splitext(os.path.basename(file_name))[0]
    cand = os.path.join(raw_dir, f"{stem}.jpg")
    return cand if os.path.exists(cand) else None

def _iter_records(obj):
    # Case A: {"image":{file_name,width,height}, "annotations":[...]}
    if isinstance(obj, dict) and ("image" in obj or "file_name" in obj or "filename" in obj):
        img = obj.get("image", {})
        fn = img.get("file_name") or obj.get("file_name") or obj.get("filename")
        W  = int(img.get("img_width") or obj.get("img_width") or img.get("width") or obj.get("width") or 512)
        H  = int(img.get("img_height") or obj.get("img_height") or img.get("height") or obj.get("height") or 512)
        anns = obj.get("annotations") or obj.get("objects") or obj.get("regions") or []
        rects=[]
        for a in anns:
            if "shape_attributes" in a and a["shape_attributes"].get("name")=="rect":
                sa=a["shape_attributes"]; rects.append({"x":sa["x"],"y":sa["y"],"width":sa["width"],"height":sa["height"]})
            elif "rect" in a: rects.append(a["rect"])
            elif "bbox" in a: rects.append(a["bbox"])
        if fn and rects:
            yield (fn,W,H,rects)
        return
    # Case B: VIA dict
    if isinstance(obj, dict) and all(isinstance(v,dict) for v in obj.values()):
        for v in obj.values():
            fn = v.get("filename") or v.get("file_name")
            W = int(v.get("img_width") or v.get("width") or 512)
            H = int(v.get("img_height") or v.get("height") or 512)
            rects=[]
            for r in v.get("regions", []):
                sa=r.get("shape_attributes",{})
                if sa.get("name")=="rect":
                    rects.append({"x":sa["x"],"y":sa["y"],"width":sa["width"],"height":sa["height"]})
            if fn and rects:
                yield (fn,W,H,rects)
        return
    # Case C: list
    if isinstance(obj, list):
        for it in obj:
            yield from _iter_records(it)

def _convert_split(split):
    raw_img = f"{BASE}/raw/{split}/images"
    raw_lbl = f"{BASE}/raw/{split}/labels_json"
    out_img = f"{BASE}/yolo/images/{split}"
    out_lbl = f"{BASE}/yolo/labels/{split}"
    os.makedirs(out_img, exist_ok=True); os.makedirs(out_lbl, exist_ok=True)

    moved=0
    for jf in tqdm.tqdm(glob.glob(f"{raw_lbl}/**/*.json", recursive=True), desc=f"Convert {split}"):
        try:
            txt = open(jf,"r",encoding="utf-8").read().strip()
            obj = json.loads(txt)
        except Exception:
            continue
        for fn,W,H,rects in _iter_records(obj):
            src = _match_jpg(raw_img, fn)
            if not src:
                continue
            base = os.path.splitext(os.path.basename(src))[0]
            dst_img = os.path.join(out_img, f"{base}.jpg")
            if not os.path.exists(dst_img):
                shutil.copy2(src, dst_img)
            lines=[]
            for r in rects:
                try:
                    x,y,w,h = _xywh_from_rect(r,W,H)
                    lines.append(_yolo_line(x,y,w,h,W,H,cls=0))
                except:
                    pass
            if lines:
                with open(os.path.join(out_lbl, f"{base}.txt"),"w") as f:
                    f.write("\n".join(lines))
                moved += 1
    print(f"[{split}] YOLO labels generated for {moved} images.")

_convert_split("train")
_convert_split("val")

print("train imgs:", len(glob.glob(f'{BASE}/yolo/images/train/*.jpg')),
      "labels:", len(glob.glob(f'{BASE}/yolo/labels/train/*.txt')))
print("val   imgs:", len(glob.glob(f'{BASE}/yolo/images/val/*.jpg')),
      "labels:", len(glob.glob(f'{BASE}/yolo/labels/val/*.txt')))


Convert train: 100%|██████████| 8052/8052 [00:01<00:00, 6920.30it/s]


[train] YOLO labels generated for 8052 images.


Convert val: 100%|██████████| 1006/1006 [00:00<00:00, 6842.20it/s]

[val] YOLO labels generated for 1006 images.
train imgs: 8052 labels: 8052
val   imgs: 1006 labels: 1006


In [5]:
# === 3. data.yaml ===
data_yaml = f"""
path: {BASE}/yolo
train: images/train
val: images/val
names:
  0: chimney
"""
with open(f"{BASE}/chimney.yaml","w") as f:
    f.write(data_yaml.strip()+"\n")
print(open(f"{BASE}/chimney.yaml").read())


path: /content/ks_chimney_yolo/yolo
train: images/train
val: images/val
names:
  0: chimney



In [ ]:
# === 4. Clean environment & pin versions (auto restart) ===
# 불필요/충돌 유발 패키지 제거
!pip -q uninstall -y \
  ultralytics opencv-python opencv-contrib-python opencv-python-headless \
  albumentations albucore fastai spacy thinc \
  cudf-cu12 dask-cudf-cu12 cuml-cu12 cupy-cuda12x cugraph-cu12 \
  torchtune bigframes datasets gcsfs fsspec \
  pandas requests jedi > /dev/null

# === 4-수정: 남은 RAPIDS 의존 패키지 제거 + 필수 조합 재설치 ===
!pip -q uninstall -y pylibcugraph-cu12 nx-cugraph-cu12 cugraph-cu12

!pip -q install --upgrade "pip<25"
!pip -q install --force-reinstall \
  "numpy==1.26.4" \
  "opencv-python-headless==4.8.1.78" \
  "ultralytics==8.2.103" \
  "pandas==2.2.2" \
  "requests==2.32.4" \
  "jedi==0.19.1" \
  pillow tqdm

import os; os.kill(os.getpid(), 9)

In [4]:
# === 5. Version check + Train/Val ===
import os, sys, subprocess, glob, textwrap, traceback
import numpy as np, cv2, pandas as pd, requests, torch, ultralytics
print("numpy :", np.__version__)          # 1.26.4
print("opencv:", cv2.__version__)         # 4.8.1.78 (다른 버전도 동작은 보통 OK)
print("pandas:", pd.__version__)          # 2.2.2
print("requests:", requests.__version__)  # 2.32.4
print("torch  :", torch.__version__)
print("ultralytics:", ultralytics.__version__)  # 8.2.103

# BASE / yaml 고정(글롭 스캔 제거)
BASE = "/content/ks_chimney_yolo"
yaml_path = f"{BASE}/chimney.yaml"
if not os.path.exists(yaml_path):
    for p in [
        f"{BASE}/yolo/images/train", f"{BASE}/yolo/images/val",
        f"{BASE}/yolo/labels/train", f"{BASE}/yolo/labels/val",
    ]:
        os.makedirs(p, exist_ok=True)
    with open(yaml_path, "w") as f:
        f.write(textwrap.dedent(f"""\
        path: {BASE}/yolo
        train: images/train
        val: images/val
        names:
          0: chimney
        """).strip()+"\n")

print("\n-- chimney.yaml --")
print(open(yaml_path).read())

# 데이터 존재 점검
def _cnt(pat): return len(glob.glob(pat))
n_tr_img = _cnt(f"{BASE}/yolo/images/train/*.jpg")
n_tr_lbl = _cnt(f"{BASE}/yolo/labels/train/*.txt")
n_va_img = _cnt(f"{BASE}/yolo/images/val/*.jpg")
n_va_lbl = _cnt(f"{BASE}/yolo/labels/val/*.txt")
print(f"\n[COUNT] train: images={n_tr_img}, labels={n_tr_lbl} | val: images={n_va_img}, labels={n_va_lbl}")

# W&B 완전 비활성화
os.environ["WANDB_DISABLED"] = "true"
os.environ["WANDB_MODE"] = "offline"
os.environ["WANDB_SILENT"] = "true"
subprocess.run([sys.executable, "-m", "pip", "uninstall", "-y", "wandb"], check=False)
from ultralytics.utils import SETTINGS
SETTINGS.update({'wandb': False})

# 학습(OOM 자동 폴백)
from ultralytics import YOLO
model = YOLO("yolov8n.pt")
device = 0 if torch.cuda.is_available() else "cpu"
print("device:", device)

def train_with_fallback(bs_init=16, epochs=100, imgsz=512, lr0=0.01, name="yolov8n_ks_train"):
    bs = bs_init
    last_err = None
    while True:
        try:
            print(f"\n[TRAIN] batch={bs}, epochs={epochs}, imgsz={imgsz}, lr0={lr0}")
            results = model.train(
                data=yaml_path,
                imgsz=imgsz,
                epochs=epochs,
                batch=bs,
                optimizer="SGD",
                lr0=lr0,
                device=device,
                project=BASE,
                name=name,
                seed=42,
                deterministic=True,
                patience=50,
                verbose=True
            )
            return results, bs
        except RuntimeError as e:
            msg = str(e).lower()
            last_err = e
            if "out of memory" in msg or ("cuda" in msg and "memory" in msg):
                if isinstance(device, int):
                    torch.cuda.empty_cache()
                if bs > 1:
                    print(f"⚠️  OOM 감지 → 배치 {bs} → {max(1, bs//2)} 로 폴백 재시도")
                    bs = max(1, bs//2)
                    continue
            print("❌ 학습 실패:")
            traceback.print_exc()
            raise last_err

results, used_bs = train_with_fallback()
print(f"\n✅ Train finished with batch={used_bs}")

# 검증(mAP@0.5)
val = model.val(data=yaml_path, imgsz=512, split="val", plots=True)
try:
    mAP50 = val.results_dict.get('metrics/mAP50(B)', val.box.map50)
except Exception:
    mAP50 = getattr(getattr(val, "box", None), "map50", None)
print("mAP@0.5:", mAP50 if mAP50 is not None else "N/A")

# 결과 경로 안내
runs_dir = os.path.join(BASE, "yolov8n_ks_train")
weights_dir = os.path.join(runs_dir, "weights")
best_pt = os.path.join(weights_dir, "best.pt")
print("\n[RESULT PATHS]")
print("runs dir   :", runs_dir)
print("weights dir:", weights_dir)
print("best model :", best_pt if os.path.exists(best_pt) else "(not found yet)")


numpy : 1.26.4
opencv: 4.11.0
pandas: 2.2.2
requests: 2.32.4
torch  : 2.8.0+cu128
ultralytics: 8.2.103

-- chimney.yaml --
path: /content/ks_chimney_yolo/yolo
train: images/train
val: images/val
names:
  0: chimney


[COUNT] train: images=8052, labels=8052 | val: images=1006, labels=1006
device: 0

[TRAIN] batch=16, epochs=100, imgsz=512, lr0=0.01
New https://pypi.org/project/ultralytics/8.3.199 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.103 🚀 Python-3.12.11 torch-2.8.0+cu128 CUDA:0 (NVIDIA A100-SXM4-80GB, 81222MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/ks_chimney_yolo/chimney.yaml, epochs=100, time=None, patience=50, batch=16, imgsz=512, save=True, save_period=-1, cache=False, device=0, workers=8, project=/content/ks_chimney_yolo, name=yolov8n_ks_train, exist_ok=False, pretrained=True, optimizer=SGD, verbose=True, seed=42, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp

100%|██████████| 755k/755k [00:00<00:00, 34.7MB/s]


Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

train: Scanning /content/ks_chimney_yolo/yolo/labels/train... 8052 images, 0 backgrounds, 0 corrupt: 100%|██████████| 8052/8052 [00:05<00:00, 1508.73it/s]


train: New cache created: /content/ks_chimney_yolo/yolo/labels/train.cache


...


100 epochs completed in 1.224 hours.
Optimizer stripped from /content/ks_chimney_yolo/yolov8n_ks_train/weights/last.pt, 6.2MB
Optimizer stripped from /content/ks_chimney_yolo/yolov8n_ks_train/weights/best.pt, 6.2MB

Validating /content/ks_chimney_yolo/yolov8n_ks_train/weights/best.pt...
Ultralytics YOLOv8.2.103 🚀 Python-3.12.11 torch-2.8.0+cu128 CUDA:0 (NVIDIA A100-SXM4-80GB, 81222MiB)
Model summary (fused): 168 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 32/32 [00:04<00:00,  7.36it/s]


                   all       1006       1322      0.983      0.977      0.992      0.827
Speed: 0.1ms preprocess, 0.3ms inference, 0.0ms loss, 0.9ms postprocess per image
Results saved to /content/ks_chimney_yolo/yolov8n_ks_train

✅ Train finished with batch=16
Ultralytics YOLOv8.2.103 🚀 Python-3.12.11 torch-2.8.0+cu128 CUDA:0 (NVIDIA A100-SXM4-80GB, 81222MiB)
Model summary (fused): 168 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /content/ks_chimney_yolo/yolo/labels/val.cache... 1006 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1006/1006 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 63/63 [00:04<00:00, 12.80it/s]


                   all       1006       1322      0.983      0.977      0.992      0.827
Speed: 0.1ms preprocess, 0.8ms inference, 0.0ms loss, 0.7ms postprocess per image
Results saved to /content/ks_chimney_yolo/yolov8n_ks_train2
mAP@0.5: 0.9916841747534443

[RESULT PATHS]
runs dir   : /content/ks_chimney_yolo/yolov8n_ks_train
weights dir: /content/ks_chimney_yolo/yolov8n_ks_train/weights
best model : /content/ks_chimney_yolo/yolov8n_ks_train/weights/best.pt


In [5]:
# === 6. Sample prediction ===
import glob, os, random
from ultralytics import YOLO

BASE = "/content/ks_chimney_yolo"
best = f"{BASE}/yolov8n_ks_train/weights/best.pt"

if os.path.exists(best):
    model = YOLO(best)
    val_imgs = glob.glob(f"{BASE}/yolo/images/val/*.jpg")
    sample = random.sample(val_imgs, k=min(8, len(val_imgs))) if val_imgs else []
    out_dir = f"{BASE}/pred_samples"
    os.makedirs(out_dir, exist_ok=True)

    if sample:
        _ = model.predict(source=sample, imgsz=512, conf=0.25,
                          save=True, project=out_dir, name="vis", exist_ok=True)
        print("Saved predictions to:", f"{out_dir}/vis")
    else:
        print("No validation images for prediction.")
else:
    print("best model not found; skip prediction.")



0: 512x512 1 chimney, 7.9ms
1: 512x512 1 chimney, 7.9ms
2: 512x512 2 chimneys, 7.9ms
3: 512x512 1 chimney, 7.9ms
4: 512x512 1 chimney, 7.9ms
5: 512x512 1 chimney, 7.9ms
6: 512x512 1 chimney, 7.9ms
7: 512x512 1 chimney, 7.9ms
Speed: 1.3ms preprocess, 7.9ms inference, 1.0ms postprocess per image at shape (1, 3, 512, 512)
Results saved to /content/ks_chimney_yolo/pred_samples/vis
Saved predictions to: /content/ks_chimney_yolo/pred_samples/vis
